# Lab 3-1: Drift Monitoring Pipeline

## Part 2: 모니터링 파이프라인

이 노트북에서는 Kubeflow Pipeline을 활용하여 **Data Drift 모니터링을 자동화**하는 방법을 학습합니다.

### 학습 목표
- Kubeflow Pipeline으로 Drift 모니터링 자동화
- MLflow에 메트릭 기록
- Alert 시스템 구축 (시뮬레이션)

### 파이프라인 구조
```
Collect Data → Detect Drift → Log Metrics → Send Alert
    ↓              ↓              ↓             ↓
  1000샘플      Drift Score    MLflow        Slack/Email
               (KS Test)       기록          시뮬레이션
```

### 소요 시간: 약 30분

---

## Step 0: KFP SDK 설치 (필수!)

⚠️ **중요**: 아래 셀을 실행한 후 **반드시 커널을 재시작**해야 합니다.
- 메뉴: **Kernel → Restart Kernel**
- 또는 단축키: `0` 두 번 연속 입력

In [ ]:
!pip install kfp==2.7.0 -q

print("="*60)
print("KFP SDK installed!")
print("Please restart the kernel: Kernel -> Restart Kernel")
print("="*60)

---

## Step 1: 환경 설정

⚠️ **커널 재시작 후 이 셀부터 실행하세요!**

본인의 사용자 번호(`USER_NUM`)를 설정합니다.

In [ ]:
# ============================================================
# TODO: Change to your USER_NUM!
# ============================================================
USER_NUM = "01"  # ex: "01", "02", ..., "11", "20"

# Auto configuration
NAMESPACE = f"kubeflow-user{USER_NUM}"
MLFLOW_TRACKING_URI = f"http://mlflow-server.kubeflow-user{USER_NUM}.svc.cluster.local:5000"

print("=" * 60)
print("  Configuration")
print("=" * 60)
print(f"  USER_NUM: {USER_NUM}")
print(f"  NAMESPACE: {NAMESPACE}")
print(f"  MLFLOW_URI: {MLFLOW_TRACKING_URI}")
print("=" * 60)

## Step 2: 라이브러리 임포트 및 버전 확인

KFP SDK를 임포트하고 버전을 확인합니다. **2.7.0 이상**이어야 합니다.

In [ ]:
from kfp import dsl
from kfp import compiler

import kfp
print(f"KFP Version: {kfp.__version__}")

version = kfp.__version__.split('.')
if int(version[0]) < 2 or (int(version[0]) == 2 and int(version[1]) < 7):
    print("Warning: KFP version is less than 2.7.0!")
    print("Please run Step 0 and restart the kernel.")
else:
    print("KFP version OK!")

---

## Step 3: Component 1 - 프로덕션 데이터 수집

프로덕션 환경에서 데이터를 수집하는 컴포넌트입니다.  
실습에서는 **시뮬레이션**으로 샘플 사이즈만 지정합니다.

### KFP Component 정의 방법
```python
@dsl.component(base_image="python:3.9-slim")
def my_component(input: int) -> int:
    # 로직 작성
    return output
```

- `@dsl.component`: 컴포넌트 데코레이터
- `base_image`: 컨테이너 이미지 지정
- **타입 힌트 필수**: 입력/출력 타입을 반드시 명시

In [ ]:
@dsl.component(base_image="python:3.9-slim")
def collect_production_data(sample_size: int = 1000) -> int:
    """Collect production data simulation"""
    print(f"Data collection simulated: {sample_size} samples")
    return sample_size

print("Component 1: collect_production_data defined")

---

## Step 4: Component 2 - Drift 감지

수집된 데이터를 **기준 데이터(Reference Data)**와 비교하여 **KS Test**를 수행하고, **Drift Score**를 계산합니다.

### KS Test (Kolmogorov-Smirnov Test)
- 두 분포가 동일한지 검정하는 통계적 방법
- **p-value < 0.05**이면 분포가 다르다고 판단 (Drift 감지)

### Drift Score 계산
```
Drift Score = (Drift가 감지된 Feature 수) / (전체 Feature 수)
```

### ⚠️ 주의사항
- `packages_to_install`로 필요한 패키지 지정
- **import문은 함수 내부에 작성** (각 컴포넌트는 독립된 컨테이너에서 실행)

In [ ]:
@dsl.component(
    base_image="python:3.9-slim",
    packages_to_install=["scikit-learn==1.3.2", "pandas==2.0.3", "numpy", "scipy"]
)
def detect_drift(sample_size: int, drift_threshold: float = 0.3) -> str:
    """Detect drift using KS test"""
    from sklearn.datasets import fetch_california_housing
    import pandas as pd
    import numpy as np
    from scipy.stats import ks_2samp
    import json
    
    print(f"Loading data for drift detection...")
    
    # Load California Housing dataset
    data = fetch_california_housing(as_frame=True)
    df = data.frame
    
    # Reference data (baseline)
    reference_data = df.sample(n=2000, random_state=42)
    print(f"Reference data: {len(reference_data)} samples")
    
    # Current data (with simulated drift on MedInc feature)
    current_data = df.sample(n=sample_size, random_state=123)
    current_data = current_data.copy()
    current_data['MedInc'] = current_data['MedInc'] * 1.5 + np.random.normal(0, 0.3, len(current_data))
    print(f"Current data: {len(current_data)} samples")
    
    # Drift detection using KS Test
    n_drifted = 0
    for col in reference_data.columns:
        _, p_value = ks_2samp(reference_data[col], current_data[col])
        if p_value < 0.05:  # Significant difference
            n_drifted += 1
    
    drift_score = n_drifted / len(reference_data.columns)
    drift_detected = drift_score > drift_threshold
    
    result = {
        'drift_detected': bool(drift_detected),
        'drift_score': float(drift_score),
        'n_drifted': int(n_drifted)
    }
    
    print(f"Drift Score: {drift_score:.2f}")
    print(f"Drifted Features: {n_drifted}/{len(reference_data.columns)}")
    print(f"Drift Detected: {drift_detected}")
    
    return json.dumps(result)

print("Component 2: detect_drift defined")

---

## Step 5: Component 3 - 메트릭 기록

계산된 Drift Score와 결과를 **MLflow에 기록**합니다.  
나중에 추적하고 시각화할 수 있습니다.

### MLflow 사용법
```python
mlflow.set_tracking_uri(uri)        # MLflow 서버 연결
mlflow.set_experiment("name")       # Experiment 설정
with mlflow.start_run():            # Run 시작
    mlflow.log_metric("key", value) # 메트릭 기록
```

In [ ]:
# MLflow URI (Pipeline parameter)
mlflow_uri_for_pipeline = MLFLOW_TRACKING_URI
print(f"MLflow URI: {mlflow_uri_for_pipeline}")

In [ ]:
@dsl.component(
    base_image="python:3.9-slim",
    packages_to_install=["mlflow==2.9.2"]
)
def log_metrics(drift_result: str, mlflow_uri: str) -> str:
    """Log metrics to MLflow"""
    import json
    import os
    
    result = json.loads(drift_result)
    
    print(f"Drift Score: {result['drift_score']:.2f}")
    print(f"Drift Detected: {result['drift_detected']}")
    print(f"Drifted Features: {result['n_drifted']}")
    
    try:
        import mlflow
        
        print(f"Connecting to MLflow: {mlflow_uri}")
        os.environ['MLFLOW_TRACKING_URI'] = mlflow_uri
        mlflow.set_tracking_uri(mlflow_uri)
        
        try:
            mlflow.set_experiment("drift-monitoring")
        except Exception as e:
            print(f"Creating new experiment...")
            mlflow.create_experiment("drift-monitoring")
            mlflow.set_experiment("drift-monitoring")
        
        with mlflow.start_run(run_name="drift-check"):
            mlflow.log_metric("drift_score", result['drift_score'])
            mlflow.log_metric("drift_detected", 1 if result['drift_detected'] else 0)
            mlflow.log_metric("n_drifted", result['n_drifted'])
            mlflow.set_tag("pipeline", "monitoring")
        
        print("Metrics logged to MLflow successfully")
        return "success"
        
    except Exception as e:
        print(f"MLflow logging failed: {e}")
        print("Continuing without MLflow...")
        return "success-no-mlflow"

print("Component 3: log_metrics defined")

---

## Step 6: Component 4 - 알림 발송

Drift가 감지되면 **알림을 발송**합니다.  
실습에서는 **콘솔 출력으로 시뮬레이션**합니다.

### 실제 환경에서는:
- Slack 알림
- Email 발송
- PagerDuty 연동

In [ ]:
@dsl.component(base_image="python:3.9-slim")
def send_alert(drift_result: str) -> str:
    """Send alert simulation"""
    import json
    
    result = json.loads(drift_result)
    
    if result['drift_detected']:
        print("ALERT: Data Drift Detected!")
        print(f"Drift Score: {result['drift_score']:.2f}")
        print("Action: Consider model retraining")
        return "alert-sent"
    else:
        print("OK: No significant drift detected")
        return "no-alert"

print("Component 4: send_alert defined")

---

## Step 7: Pipeline 정의

위에서 정의한 **4개의 컴포넌트를 연결**하여 파이프라인을 구성합니다.

### 파이프라인 흐름
```
1. collect_task: 프로덕션 데이터 수집 (시뮬레이션)
2. detect_task: Drift 감지 (KS Test 수행)
3. log_task: MLflow에 메트릭 기록
4. alert_task: 알림 발송 (시뮬레이션)
```

### 핵심 개념: `.output`으로 데이터 전달
```python
step1 = component1(input=param)
step2 = component2(input=step1.output)  # step1의 출력을 step2의 입력으로!
```

In [ ]:
@dsl.pipeline(
    name="drift-monitoring",
    description="drift monitoring pipeline"
)
def drift_monitoring_pipeline(
    sample_size: int = 1000,
    drift_threshold: float = 0.3,
    mlflow_uri: str = mlflow_uri_for_pipeline
):
    """Drift monitoring pipeline"""
    
    # Step 1: Collect data (simulated)
    collect_task = collect_production_data(sample_size=sample_size)
    
    # Step 2: Detect drift
    detect_task = detect_drift(
        sample_size=collect_task.output,
        drift_threshold=drift_threshold
    )
    
    # Step 3: Log metrics to MLflow
    log_task = log_metrics(
        drift_result=detect_task.output,
        mlflow_uri=mlflow_uri
    )
    
    # Step 4: Send alert
    alert_task = send_alert(drift_result=detect_task.output)

print("Pipeline defined!")

---

## Step 8: Pipeline 컴파일

정의된 파이프라인을 **YAML 파일로 컴파일**합니다.  
이 YAML 파일을 Kubeflow UI에 업로드하여 실행할 수 있습니다.

In [ ]:
pipeline_filename = 'drift_monitoring_pipeline.yaml'

compiler.Compiler().compile(
    pipeline_func=drift_monitoring_pipeline,
    package_path=pipeline_filename
)

print()
print("=" * 60)
print("  Pipeline compiled successfully!")
print("=" * 60)
print()
print(f"  Output file: {pipeline_filename}")
print()
print("  Next steps:")
print("    1. Upload pipeline to Kubeflow UI")
print("    2. Click 'Create Run'")
print("    3. Set parameters and Start")
print("=" * 60)

---

## Step 9: 생성된 YAML 파일 확인

In [ ]:
with open(pipeline_filename, 'r') as f:
    lines = f.readlines()
    print(f"File: {pipeline_filename} (first 30 lines)")
    print("=" * 60)
    for i, line in enumerate(lines[:30]):
        print(f"{i+1:3d} | {line}", end='')
    if len(lines) > 30:
        print(f"\n... ({len(lines)} lines total)")

---

## 🎯 Kubeflow UI에서 실행하기

### Step 1: Pipeline 업로드
1. Kubeflow UI 접속
2. 왼쪽 메뉴에서 **"Pipelines"** 클릭
3. 오른쪽 상단의 **"+ Upload pipeline"** 버튼 클릭
4. `drift_monitoring_pipeline.yaml` 파일 선택
5. **"Create"** 버튼 클릭

### Step 2: Run 생성
1. 업로드된 파이프라인 클릭
2. **"+ Create run"** 버튼 클릭
3. Parameters 설정:
   - `sample_size`: 1000 (기본값)
   - `drift_threshold`: 0.3 (기본값)
   - `mlflow_uri`: 자동 입력됨
4. **"Start"** 버튼 클릭

### Step 3: 실행 결과 확인
- **Graph 탭**에서 각 컴포넌트 실행 상태 확인
- 각 컴포넌트 클릭 → **Logs** 탭에서 로그 확인

### 예상 결과
```
✅ collect-production-data: Succeeded
✅ detect-drift: Succeeded  
✅ log-metrics: Succeeded
✅ send-alert: Succeeded
```

---

## ✅ Part 2 완료 체크리스트

- [ ] KFP SDK 2.7.0 이상 설치 완료
- [ ] USER_NUM 설정 완료
- [ ] `drift_monitoring_pipeline.yaml` 파일 생성 완료
- [ ] Kubeflow UI에 파이프라인 업로드 완료
- [ ] 파이프라인 실행 성공
- [ ] 4개 컴포넌트 모두 녹색 체크 확인

---

## 📚 핵심 개념 정리

### 1. KFP Component 정의
```python
@dsl.component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas", "numpy"]
)
def my_component(input_value: int) -> str:
    import pandas as pd  # 함수 내부에서 import!
    result = str(input_value * 2)
    return result
```

### 2. Pipeline 정의
```python
@dsl.pipeline(name="my-pipeline")
def my_pipeline(param1: int = 10):
    step1 = component1(input=param1)
    step2 = component2(input=step1.output)  # .output으로 연결!
```

### 3. 컴파일
```python
compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path='my_pipeline.yaml'
)
```

---

## 🔜 다음 단계

**Part 3: Auto-Retraining Pipeline**에서는 Drift Score가 임계값을 넘으면 자동으로 재학습하는 파이프라인을 만듭니다.